# Disagreement between Jury and Public Votes in Eurovision (2016-2019)

## Introduction
Since 2016, Eurovision has used the [dual voting system](https://eurovision.tv/about/voting) of jury and public votes (the latter commonly known as the "televote"). Both the jury and public independently vote on their top 10 songs and allocate points as follows:
* The highest-voted country is awarded 12 points.
* The second highest-voted country is awarded 10 points.
* The third to tenth highest-voted countries are awarded 8, 7, ..., 1 points respectively. 
* The jury and public cannot vote for their own countries.

In other words, the jury awards \[12, 10, 8, 7, 6, 5, 4, 3, 2, 1\] points, and the televote awards another \[12, 10, 8, 7, 6, 5, 4, 3, 2, 1\] points. A country's jury and public may not vote for its own country. The country that receives the highest points is the winner of that year's Eurovision, and gain the right to host the contest in their country (except for Australia, who has to choose a European country to co-host with).

Per the official Eurovision Jury voting guidelines, juries are asked to rate each country based on the following qualities:
* vocal capacity of the artist(s), 
* the performance on stage,
* the composition and originality of the song, and 
* the overall impression by the act.

There is no general guideline for the public when casting their votes (beside not being able to vote for their own country).

## Motivation

However, it is well known that Eurovision is not merely a contest that judges each song's popularity, but also one with subtle political undertones, most obviously hinted due to the fact that each artist must explicitly represent a country. While the European Broadcasting Union (EBU) explicitly forbids political expression in the contest, there are nonetheless less obvious ways of highlighting international relations during the contest.

Voting is one way to reflect such politics. Country A may vote for Country B due for political reasons, such as similar cultures, languages, history, or some elements in the song itself. Countries are often accused of "bloc voting" or "collusive voting", the act of voting for certain countries regardless of the quality of their song, a [well-documented](http://jasss.soc.surrey.ac.uk/9/2/1.html) occurance.

If we consider only the quality of a song and public reception to the song as voting factors, then agreement between the public and jury votes may suggest that the public has the ability to impartially determine the quality of a song per EBU standards, whereas disagreements may suggest that the jury are a more objective method of awarding points that the televote.

## Objectives
Regardless of the implications of public and jury voting, it is necessary to first compute the voting differences of each party. 

This notebook will contain the following:

1. Read and clean the voting data of the 2016-2019 Eurovision contest's grand final
2. Calculate the **disagreement value** between each country's jury and public votes
3. Plot each country's disagreement values
4. Show results and discoveries.

## 1. Reading and cleaning data

The data is obtained from [datagraver](https://data.world/datagraver/eurovision-song-contest-scores-1975-2019/workspace/file?filename=eurovision_song_contest_1975_2019v5.xlsx), with some incorrect information edited out, such as Belarus giving Russia both 12 (correct) and 0 (incorrect) televote points in the 2019 grand final.

### Import modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import itertools

# import common functions
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/modules")

import functions as common_functions


### Read data

In [ ]:
# original:https://data.world/datagraver/eurovision-song-contest-scores-1975-2019/workspace/file?filename=eurovision_song_contest_1975_2019v5.xlsx
# note that the data for 2019 Final: Belarus has been corrected by removing incorrect score reports. In addition, 2018f's "The Netherands" has been corrected to "The Netherlands".
csv = pd.read_csv(".\eurovision_song_contest_1975_2019v5.csv")

csv.head()

### Clean / modify CSV data

In [ ]:

# rename columns
csv = csv.rename(columns={"Jury or Televoting" : "Jury_or_Televoting", "From country": "From_country", "To country" : "To_country"})

years = [2016, 2017, 2018, 2019]

def grand_final_votes(csv, year: int):
    # picking only relevant data
    grand_final = csv[(csv.Year == year) & (csv["(semi-) final"] == "f") & (csv["Duplicate"] != "x") & (csv["From_country"] != csv["To_country"])]
    grand_final = grand_final[ ["Jury_or_Televoting", "From_country", "To_country", "Points"] ]

    # check for faulty data
    duplicated = grand_final[grand_final.duplicated( ["Jury_or_Televoting", "From_country", "To_country"] )]
    assert len(duplicated) == 0, f"({year}) Voting has duplicates, check CSV for faulty data"

    # note: this is to ensure each vote is unique.
    juryvotes = grand_final[grand_final["Jury_or_Televoting"] == "J"].drop_duplicates()
    televotes = grand_final[grand_final["Jury_or_Televoting"] == "T"].drop_duplicates()

    # join jury and televote data into one table
    voting_pairs = juryvotes.join(
        televotes.set_index(["From_country", "To_country"]), 
        on=["From_country", "To_country"], 
        lsuffix="_Jury", 
        rsuffix="_Televote"
    )
    voting_pairs = voting_pairs.drop(["Jury_or_Televoting_Jury", "Jury_or_Televoting_Televote"], axis=1)

    assert len(juryvotes) == len(televotes) and len(televotes) == len(
        voting_pairs), f"({year}) Voting data not consistent, check CSV for faulty data"
    return voting_pairs

# maps year to voting
voting_by_year = {year : grand_final_votes(csv, year) for year in years}



## 2. Disagreement value

Here we calculuate the disagreement between each the country's jury and public votes.
* A **high** disagreement value means that the jury and the public voted differently, such as ranking countries differently or voting for different countries altogether.
* A **low** disagreement value means that the jury and the public voted similarly, i.e. they agree.

### Formal definition

* The set of all countries that voted in a certain year is $C$.
* A country $c \in C$ has two sets of votes:
    * The _televote_, $T_c = \{12, 10, 8, 7, 6, 5, 4, 3, 2, 1, 0\}$. 0 means the public awarded 0 points to $d$.
    * The _jury vote_, $J_c = \{12, 10, 8, 7, 6, 5, 4, 3, 2, 1, 0\}$. 0 means the jury awarded 0 points to $d$.
* The votes in $T_c$ and $J_c$ are given to countries in $D = C - c$, i.e. $c$ cannot vote for itself.
    * $c$ giving $x \in T_c$ public votes to $d$ is written as $T(c, d) = x$.
    * $c$ giving $x \in J_c$ jury votes to $d$ is written as $J(c, d) = x$.
* $T_c$ may not award multiple elements to the same country $d$, i.e. $d$ cannot recieve both 12 and 5 points from $c$'s televote.
* $J_c$ may not award multiple elements to the same country $d$, i.e. $d$ cannot recieve both 12 and 5 points from $c$'s jury vote.

The **disagreement** between a country $c$'s jury and public votes for country $d$ is defined as:

$$dis(c, d) = \sum_{(c, d) \in C ⨉ D} | T(c, d) - J(c, d)| $$

When considering a graph plotting a country's jury votes (x-axis) vs. its public votes (y-axis), $dis(c,d)$ may be seen as the total distance of the points $\{ (T(c, d), J(c, d)) : (c, d) \in C ⨉ D \}$ to the "agreement line" $y = x$ (that is, televote points = jury points), divided by a constant of $\sqrt{2}$.

This is because any point $(a, b)$'s distance to the line $y = x$ is $\frac{|b-a|}{\sqrt{2}}$, and thus the sum of the distances of all jury vs. televote points to the agreement line is  

$$ \frac{\sum_{(c, d) \in C ⨉ D} | T(c, d) - J(c, d)|}{\sqrt{2}}. $$

Nevertheless, we use $dis(c,d)$ as the $(\sqrt{2})^{-1}$ factor is constant.

#### Observations
* The highest possible disagreement value is 116, i.e. the public gave points to 10 countries and the jury gave points to 10 _other_ countries.
* The lowest possible disagreement value is 0, i.e. the public and jury gave the exact same points to the exact same countries.


In [ ]:
# Generate disagreement value (flat and weighted)
for year, voting in voting_by_year.items():
    voting["disagree_val"] = np.abs(voting["Points_Jury"] - voting["Points_Televote"]) 

voting_by_year[2019].head() # sample disagreement value

Here, we also consider a related problem: people tend to view 12 (and other higher) points to be significantly more imporant than lower points. In other words, higher points may be weighted significantly higher than lower points, something we can factor into our disagreement value. 

Specifically, the _weight function_ $w$ simply maps the a score $k$ (from 1 to 12) to a different value. In this case: $$w(k) = k^3$$

The choice of $w(k)$ is arbitrary, but in general the function should satisfy $w(k) \ge k$.

With this in mind, we can slightly modify our disagreement function to create a _weighted disagreement_ function:

$$weighted\_dis(c, d) = \sum_{(c, d) \in C ⨉ D} | w(T(c, d)) - w(J(c, d))| $$



In [ ]:
for year, voting in voting_by_year.items():
    voting["disagree_val_weighted"] = np.abs(
        (voting["Points_Jury"])**3 - (voting["Points_Televote"])**3
    )
    MAX_WIGHTED_DISAGREEMENT = 2 * ((12)**3 + (10)**3 + np.sum( (np.arange(1, 8+1))**3 ))

voting_by_year[2019].head()

In this case, we can see that when higher points are weighted more, the order of each country's disagreement changes. This is because drastic differences in voting are much more heavily accentuated. For example, the disagreement value between 1 and 12 is $12 - 1 = 11$; the weighted disagreement value is $12^3 - 1^3 = 1727$.

That is, if the jury and public give many similar votes, e.g. $(2,3), (3,5), (7,9), (11,12)$, the disagreement is only $1+2+2+1=6$.
However, if the jury and public gives drastically different points, i.e. $(12,9),(11,8)$, even though the disagreement is also $3+3=6$, the weighted disagreement value is $(12^3-9^3) + (11^3 - 8^3) = 1818$.

Thus, weighting points to make their differences more accentuated will cause the overall order of countries to favour countries who had overall smaller differences in ranking a country, and penalise countries whose juries and public had larger differences in ranking a country. 

In [ ]:
# A tabulation of the disagreement values of each country.
def disagreement_tabulate(voting):
    voting_pairs_grouped = voting.groupby(["From_country"])
    voting_pairs_grouped_summary = voting_pairs_grouped[["disagree_val", "disagree_val_weighted"]].sum()
    voting_pairs_grouped_summary_normalized = voting_pairs_grouped_summary / np.array([116, MAX_WIGHTED_DISAGREEMENT]) * 100
    return voting_pairs_grouped_summary_normalized.sort_values(by="disagree_val")
    
disagreement_by_years = { year : disagreement_tabulate(voting_by_year[year]) for year in years }

## 3. Plotting each country's disagreement

In [ ]:
# reduce text clutter by shortening names. These choices are completely arbitrary.
PLOT_ANNOTE_MAPPING = {
    "Czech Republic" : "Czechia",
    "North Macedonia": "N. MK",
    "The Netherlands": "Netherlands",
    "United Kingdom": "UK"
}

def plot_disagreement(dataframe, country_name="Malta", year=2019):
    # obtain a certain country's information
    just_one = dataframe[dataframe.From_country == country_name]
    just_one_reindexed = just_one.set_index("To_country")[["Points_Jury", "Points_Televote"]] # for plotting convenience

    # plot graph
    cmap = cm.get_cmap('Spectral')
    ax = just_one_reindexed.plot(
        kind="scatter", x="Points_Jury", y="Points_Televote", 
        c=range(len(just_one_reindexed)), colormap=cmap, s=30, figsize=(10, 5)
    )

    # annotate country names
    for k, v in just_one_reindexed.iterrows():
        if v.Points_Jury == 0 and v.Points_Televote == 0: continue # ignore countries that were not given points
        annote_text = PLOT_ANNOTE_MAPPING[k] if (k in PLOT_ANNOTE_MAPPING) else k
        ax.annotate(annote_text, v,
                    xytext=(5,-5), textcoords='offset points',
                    family='sans-serif', fontsize=8, color='darkslategrey')

    ax.axline((0, 0), (1,1))

    # unweighted: 116 is max possible disagreement, i.e. 2 * (1 + 2 + ... + 8 + 10 + 12)
    total_disagreement = '{0:.2g}%'.format(100*just_one.disagree_val.sum() / 116)
    # weighted: MAX_WIGHTED_DISAGREEMENT is max possible disagreement
    total_disagreement_weighted = '{0:.3g}%'.format(100*just_one.disagree_val_weighted.sum() / MAX_WIGHTED_DISAGREEMENT)
    ax.set_title(f"Jury-Televote Disagreement\n{country_name} ({year}) \n(Disagreement (flat) = {total_disagreement})\n(Disagreement (weighted) = {total_disagreement_weighted})")
    plt.show()

## 4. Results and discoveries

Note: Since San Marino's televotes are not determined by the public (!), we do not consider San Marino's disagreement points.

### Lowest disagreement values by year:

| Year | Country   |   disagreement (unweighted) |   disagreement (weighted) |
|:---------------|---------------:|------------------------:|------------------------:|
| 2016  | Azerbaijan     |        20.6897 |                 13.7177 |
| 2017 | Cyprus         |        25.8621 |                 23.7823 |
| 2018 | Albania        |        31.0345 |                 18.34   |
| 2019 | Cyprus         |        32.7586 |                 15.6809 |

### Highest disagreement values by year:

| Year | Country   |   disagreement (unweighted) | disagreement (weighted) |
|:---------------|---------------:|------------------------:|------------------------:|
| 2016 | Czech Republic |        91.3793 |                 99.1302 |
| 2017 | Bulgaria       |        86.2069 |                 96.2227 |
| 2018 | Poland, Israel |        89.6552 |                 99.1054 |
| 2019 | Hungary        |        82.7586 |                 97.5149 |

In [ ]:
for year in years:
    # Since San Marino's televotes are not determined by the public, drop it
    disagreement = disagreement_by_years[year].drop("San Marino")

    # compute & plot lowest disagreement
    lowest = disagreement[disagreement.disagree_val == disagreement.disagree_val.min()].iloc[0].name
    print(f"lowest in {year}: {lowest}")
    plot_disagreement(voting_by_year[year], lowest, year)

    # compute & plot highest disagreement
    highest = disagreement[disagreement.disagree_val == disagreement.disagree_val.max()].iloc[0].name
    print(f"highest in {year}: {highest}")
    plot_disagreement(voting_by_year[year], highest, year)